In [1]:
%pylab inline
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from recomendation import SvdDecomposition

Populating the interactive namespace from numpy and matplotlib


### Оригинальный код
https://github.com/alabid/PySVD/blob/master/regularizedSVD.py

### Подготовливаем обучающую и контрольную выборки

In [2]:
def read_data(path):
    df=pd.read_csv(path,sep='\t',header=None)
    df.columns=[['userid', 'itemid', 'score','hash']]
    df=df.ix[:,:3].sort_values(by=['userid', 'itemid'])
    return df
data=read_data("ua.base")
test_data=read_data("ua.test")
data.head()

,userid,itemid,score
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


### Функции кодировки/декодирования данных
Алгоритму необходимо подавать закодированные данные <br> Например: пользователи [Петя, Вася, Маша] должны быть преобразованы в [0,1,2]


In [3]:
def fit_encoder(arr):
    encoder=LabelEncoder()
    encoder.fit(arr)
    return len(arr),encoder

def encode_data(data,user_encoder,item_encoder,encode_users=False,encode_items=False,sort_vals=False,return_decoder=False):
    if encode_users:
        data['userid']=user_encoder.transform(data['userid'])
        users_decoder=lambda arr:users_encoder.inverse_transform(arr)
    else:users_decoder=None

    if encode_items:
        data['itemid']=item_encoder.transform(data['itemid'])
        items_decoder=lambda arr:items_encoder.inverse_transform(arr)     
    else:items_decoder=None
        
    if sort_vals:data=data.sort_values(by=['userid','itemid'])
    if return_decoder:
        return data,users_decoder,items_decoder
    else:
        return data

In [4]:
# получаем число уникальных пользователей и кодировщик пользователей
n_users,user_encoder=fit_encoder(pd.concat([data,test_data],axis=0).userid.unique())
# получаем число уникальных товаров(предеметов) и кодировщик предметов
n_items,item_encoder=fit_encoder(pd.concat([data,test_data],axis=0).itemid.unique())
n_users,n_items

(943, 1682)

In [5]:
#кодируем исходные данные и получаем декодеры
data,users_decoder,items_decoder=encode_data(data,user_encoder,item_encoder,True,True,sort_vals=True,return_decoder=True)
test_data=encode_data(test_data,user_encoder,item_encoder,True,True,sort_vals=True)

### Классическое разложение
Исходную матрицу оценок приблизаем **U*V**,где **U**- матрица пользователей размера **n_user x rank**,
матрица **V** -матрица товаров размера **n_items x rank**


* **rank** -ранг , **если early_stop**=True,то заканчиваем изменение весов матриц(если изменение весов следующего  раногового столбца матриц U и V не приводит к уменьшению ошибки на первой итерации)
* **regularizer** коэффициент регуляризации, подбирать можно по кросс валидации
* **lrate** -learning rate, подбирать можно по кросс валидации
* **alpha** - коэффициент адаптации learning rate ,если градиентный шаг уменьшил ошибку, lrate=lrate(1+alpha) и наоборот
* **n_iterations** -число итераций
* **warm_start** если True, исользует **U и V** из предыдущего запуска алгоритма
* **print_steps** вывод информации по итерационным шагам

In [6]:
%%time
svd=SvdDecomposition(data,n_users,n_items)
svd.train_ratings(rank=30,lrate=0.035,alpha=0.05,regularizer=0.01,
                      stop_criteria = 0.001,n_iterations = 30,trunc_score_rule='default',
                     early_stop=False,minibatch_size=0.1,warm_start=False,print_steps=True)

k= 0
iteration= 0 ; train_err= 1.04323113766
iteration= 1 ; train_err= 1.02208285839
iteration= 2 ; train_err= 1.01511446436
iteration= 3 ; train_err= 1.01006386475
iteration= 4 ; train_err= 1.00592979066
iteration= 5 ; train_err= 1.0025854088
iteration= 6 ; train_err= 0.999882355871
iteration= 7 ; train_err= 0.997678336933
iteration= 8 ; train_err= 0.995859741165
iteration= 9 ; train_err= 0.994335586914
iteration= 10 ; train_err= 0.993027667767
iteration= 11 ; train_err= 0.991882793168
iteration= 12 ; train_err= 0.990867745715
iteration= 13 ; train_err= 0.989959852889
k= 1
iteration= 0 ; train_err= 0.950498113814
iteration= 1 ; train_err= 0.924214483163
iteration= 2 ; train_err= 0.917417212936
iteration= 3 ; train_err= 0.914507868321
iteration= 4 ; train_err= 0.912722414854
iteration= 5 ; train_err= 0.911451805807
iteration= 6 ; train_err= 0.910489775636
k= 2
iteration= 0 ; train_err= 0.902114380489
iteration= 1 ; train_err= 0.897077672174
iteration= 2 ; train_err= 0.894231505673
iter

In [7]:
%%time
print ("rmsetrain: ", svd.calc_rmse(data))
print ("rmsetest: ", svd.calc_rmse(test_data))

rmsetrain:  0.83516039288
rmsetest:  0.94573726649
Wall time: 6.16 s


### Стохастический градиентный спуск (Mini batch)
не очень уверен в реализации 
* **minibatch_size** если <1 , берет соотвутсвующую долю из обучающей выборки, если >=1 , то беретсся соответсвующее число элементов из обучающей выборки

In [8]:
%%time
mini=SvdDecomposition(data,n_users,n_items)
mini.train_ratings(rank=30,lrate=0.035,regularizer=0.01,
                      stop_criteria = 0.001,n_iterations = 30,trunc_score_rule='default',
                     early_stop=False,minibatch_size=1000,warm_start=False,algorithm='minibatch')

minibatch_size= 1000
max_rank= 28
Wall time: 26.5 s


In [9]:
%%time
print ("rmsetrain: ", mini.calc_rmse(data))
print ("rmsetest: ", mini.calc_rmse(test_data))

rmsetrain:  0.933585410913
rmsetest:  0.975090292096
Wall time: 6.15 s
